In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from pyomo.opt import SolverFactory

from pyomo_models.base import BaseModel
from pyomo_models.multi_uavs import MultiUavModel
from util.scenario import Scenario
from simulate.simulate import Simulation
from util import constants
from tqdm.notebook import tqdm
from itertools import product
from datetime import datetime

# Helper functions

In [ ]:
def get_scenario(seqs: list, charging_station_positions: list):
    """
    Converts a list of 2-d numpy arrays of flight sequences in a charge scheduling scenario
    """
    doc = {}
    drones = []
    for seq in seqs:
        waypoints = []
        for x, y, z in seq:
            waypoints.append(dict(
                x=float(x),
                y=float(y),
                z=float(z),
            ))
        drones.append(dict(waypoints=waypoints))
    doc['drones'] = drones

    cs = []
    for x, y, z in charging_station_positions:
        cs.append(dict(
            x=float(x),
            y=float(y),
            z=float(z),
        ))
    doc['charging_stations'] = cs
    return Scenario(doc)

In [ ]:
def timed(func):
    def wrapper(*args, **kwargs):
        t_start = datetime.now()
        res = func(*args, **kwargs)
        elapsed = datetime.now() - t_start
        return elapsed, res

    return wrapper

# Benchmark on optimization

In [ ]:
drone_counts = [1,3]
waypoint_count = [5, 10, 15]
station_count = [1,2]

scenario_sizes = [(d,w,s) for d,w,s in product(drone_counts, waypoint_count, station_count)]

In [ ]:
# prepare scenarios
waypoint_templ = [
    [(1.0, 0.0, 0), (0.0, 1.0, 0), (-1.0, 0.0, 0), (-0.0, -1.0, 0)],
    [(0.866025403784, 0.5, 0), (-0.5, 0.866025403784, 0), (-0.866025403784, -0.5, 0), (0.5, -0.866025403784, 0)],
    [(0.5, 0.866025403784, 0), (-0.866025403784, 0.5, 0), (-0.5, -0.866025403784, 0), (0.866025403784, -0.5, 0)]
]

scenarios = []

for N_d, N_w, N_s in scenario_sizes: 
    if N_s > N_d:
        # ignore scenarios with more charging stations that UAVs
        continue
    seqs = []
    for d in range(N_d):
        templ_size = len(waypoint_templ[d])
        seq = []
        for w in range(N_w):
            seq.append(waypoint_templ[d][w % templ_size])

        seqs.append(seq)

    cs = []
    for s in range(N_s):
        cs.append((0,0,0))
    sc = get_scenario(seqs, cs)
    scenarios.append(sc)

In [ ]:
# PARAMETERS
B_min = 0.1
B_max = 1
B_start = [1, 1, 1]
v = [1,1,1]
r_charge = np.array([0.15, 0.15, 0.15])
r_deplete = np.array([0.2, 0.2, 0.2])
parameters = {p: eval(p) for p in ['B_min', 'B_max', 'B_start', 'v', 'r_charge', 'r_deplete']}
parameters['epsilon'] = 0.1

In [ ]:
t_models = []
t_solves = []
models = []


for sc in tqdm(scenarios):   
    @timed 
    def create_model(sc, parameters):
        return MultiUavModel(sc, parameters)
    
    @timed
    def solve(model): 
        # solver = SolverFactory("mindtpy")
        # return solver.solve(model, mip_solver='gurobi', nlp_solver='ipopt', tee=True)
        solver = SolverFactory("gurobi")
        solver.options['TimeLimit'] = 120
        try:
            res = solver.solve(model)
        except ValueError as e:
            print(f"failed to solve problem: {e}")
            res = None
        return res


    print(f"({sc.N_d}, {sc.N_s}, {sc.N_w})")
    
    t_model, model = create_model(sc, parameters)
    model_name = f"({sc.N_d},{sc.N_s},{sc.N_w})"
    model.name = model_name
    t_models.append(t_model)
    models.append(model)
    print(f"# constraints: {model.nconstraints():,}")
    print(f"# variables: {model.nvariables():,}")
    print(f"# objectives: {model.nobjectives():,}")
    print(f"t_model: {t_model}")
    
    t_solve, solution = solve(model)    
    t_solves.append(t_solve)
    print(f"t_solve: {t_solve}")
    print()

In [ ]:
for i, m in enumerate(models):
    max_time = m.execution_time()
    if not max_time:
        continue
    
    fig, axes = plt.subplots(m.N_d, 1, figsize=(max(2, max_time/20), m.N_d), dpi=80, sharey=True)
    fig.subplots_adjust(hspace=.4)
    for d in range(m.N_d):
        if m.N_d > 1:
            ax = axes[d]
        else:
            ax = axes
            
        sim = Simulation.from_base_model(m, d)
        sim.plot_charge(ax=ax)
        
        # set xtick
        t = sim.simulate()[1][-1]
        ax.set_xticks([t], [f"{t:.2f}s"])
        
        ax.set_xlim([-1, max_time+1])  
        if d != sc.N_d - 1:
            ax.set_xlabel("")
    fig.suptitle(m.name)

# GurobiPersistent

In [ ]:
t_models = []
t_solves = []
models = []


for sc in tqdm(scenarios):   
    @timed 
    def create_model(sc, parameters):
        return MultiUavModel(sc, parameters)
    
    @timed
    def solve(model): 
        # solver = SolverFactory("mindtpy")
        # return solver.solve(model, mip_solver='gurobi', nlp_solver='ipopt', tee=True)
        solver = SolverFactory("gurobi_persistent")
        solver.options['TimeLimit'] = 120
        solver.set_instance(model)
        try:
            res = solver.solve()
        except ValueError as e:
            print(f"failed to solve problem: {e}")
            res = None
        return res


    print(f"({sc.N_d}, {sc.N_s}, {sc.N_w})")
    
    t_model, model = create_model(sc, parameters)
    model_name = f"({sc.N_d},{sc.N_s},{sc.N_w})"
    model.name = model_name
    t_models.append(t_model)
    models.append(model)
    print(f"# constraints: {model.nconstraints():,}")
    print(f"# variables: {model.nvariables():,}")
    print(f"# objectives: {model.nobjectives():,}")
    print(f"t_model: {t_model}")
    
    t_solve, solution = solve(model)    
    t_solves.append(t_solve)
    print(f"t_solve: {t_solve}")
    print()